In [1]:
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.neural_network import MLPClassifier

import pandas as pd
import re
import numpy as np
import scipy
import copy
import re

In [2]:
master_data = pd.read_csv("multiclass.csv")
# master_data = pd.read_csv("train.csv")

In [3]:
features = np.array(master_data.iloc[:, 1])

In [4]:
labels = np.array(master_data.iloc[:, 0])

In [5]:
test = features
p_data = []

for eachPoint in test:
    p_data.append([re.sub(r"(http|ftp|https)\S+", "", str(eachPoint))])

In [6]:
test = p_data

In [7]:
def lowercase(arr):
    return np.char.lower(arr)

In [8]:
def removeStopWords(arr):
    stopWords = set(stopwords.words('english'))
    
    filteredWords = []
    for a in arr:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            if w not in stopWords:
                wordsFiltered = wordsFiltered + " " + w
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [9]:
def removeSymbols(arr):
    symbols = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
    for i in range(len(symbols)):
        arr = np.char.replace(arr, symbols[i], '')   
    return np.char.replace(arr, "  ", " ")

In [10]:
def stemming(word):
    porter_stemmer=PorterStemmer()
    filteredWords = []
    for a in word:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            wordsFiltered = wordsFiltered + " " + porter_stemmer.stem(w)
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [11]:
def removeImplicitNoise(arr):
    return np.char.replace (arr, 'NEWLINE_TOKEN', '')

In [12]:
def processClean(data):
    a = removeImplicitNoise(data)
    a = lowercase(a)
    a = removeStopWords(a)
    a = removeSymbols(a)
    a = stemming(a)
    return a

a = processClean(test)

In [13]:
list_l = []

for i in a:
    list_l.append(str(i[0]))

In [14]:
def make_grams(data, n = 3):
    
    grammed_data = []
    
    for i in data:

        k = copy.deepcopy(i)

        for r in range(2,n+1):
            sixgrams = ngrams(i.split(), r)

            for grams in sixgrams:
                g = ""
                for p in grams:
                    g = g+p

                k = k+" "+g
        grammed_data.append(k)
    
    return grammed_data

grammed_data = make_grams(list_l)

In [15]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(feature_vector.shape)

(4500, 151969)


In [16]:
# TF-IDF
vectorizer = TfidfVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(vector.shape)

(4500, 151969)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, test_size=0.25, random_state=25)

In [ ]:
parameters={
'hidden_layer_sizes': [(20,30), (30,20,1)],
'activation': ["logistic", "relu", "Tanh"]
}

In [ ]:
clf = GridSearchCV(MLPClassifier(),parameters,n_jobs=-1, cv=2)

In [ ]:
clf

In [ ]:
clf = MLPClassifier()
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
clf.score(X_train, y_train)